In [0]:
!rm -r ee-js-to-python-syntax earthengine-api

In [85]:
# Get the package
!git clone https://github.com/gee-community/ee-js-to-python-syntax.git

Cloning into 'ee-js-to-python-syntax'...
remote: Counting objects: 18, done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 18 (delta 5), reused 14 (delta 4), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [86]:
# See the comandline options
%run ee-js-to-python-syntax/eejs2python/main -h


usage: main.py [-h] [-dir JS_DIR] [-out OUT_DIR] [-fm ADD_FRONT_MATTER]
               [-cm CONVERT_MAP]

optional arguments:
  -h, --help            show this help message and exit
  -dir JS_DIR           Path to the directory containing JavaScript files
  -out OUT_DIR          Path to the directory for saving converted files, this
                        will be created if not available
  -fm ADD_FRONT_MATTER, --add-front-matter ADD_FRONT_MATTER
                        Add standard earthengine Python front matter
  -cm CONVERT_MAP, --convert-map CONVERT_MAP
                        Convert JS Map. to Python syntax


In [87]:
# Get some examples
!git clone https://github.com/google/earthengine-api.git

Cloning into 'earthengine-api'...
remote: Counting objects: 4010, done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 4010 (delta 14), reused 47 (delta 13), pack-reused 3939
Receiving objects: 100% (4010/4010), 10.17 MiB | 31.10 MiB/s, done.
Resolving deltas: 100% (2463/2463), done.


In [88]:
# See which examples are available
!ls earthengine-api/javascript/src/examples/Image

CannyEdgeDetector.js		  Hillshade.js		   PixelArea.js
CenterPivotIrrigationDetector.js  HoughTransform.js	   PixelLonLat.js
Clamp.js			  HSVPanSharpening.js	   Polynomial.js
ConnectedPixelCount.js		  LandcoverCleanup.js	   ReduceRegion.js
Expression.js			  ModisQaBands.js	   ZeroCrossing.js
FromName.js			  NormalizedDifference.js


In [89]:
%run main.py -dir earthengine-api/javascript/src/examples/Image/ -out converted_python -fm True


Conversion written to  <_io.TextIOWrapper name='converted_python/ReduceRegion.py' mode='w' encoding='UTF-8'>
Conversion written to  <_io.TextIOWrapper name='converted_python/NormalizedDifference.py' mode='w' encoding='UTF-8'>
Conversion written to  <_io.TextIOWrapper name='converted_python/ConnectedPixelCount.py' mode='w' encoding='UTF-8'>
Conversion written to  <_io.TextIOWrapper name='converted_python/FromName.py' mode='w' encoding='UTF-8'>
Conversion written to  <_io.TextIOWrapper name='converted_python/PixelArea.py' mode='w' encoding='UTF-8'>
Conversion written to  <_io.TextIOWrapper name='converted_python/ModisQaBands.py' mode='w' encoding='UTF-8'>
Conversion written to  <_io.TextIOWrapper name='converted_python/HoughTransform.py' mode='w' encoding='UTF-8'>
Conversion written to  <_io.TextIOWrapper name='converted_python/Polynomial.py' mode='w' encoding='UTF-8'>
Conversion written to  <_io.TextIOWrapper name='converted_python/Clamp.py' mode='w' encoding='UTF-8'>
Conversion written

In [90]:
# Check out the conversion
# Note conversion of Map.addLayer ect is still to be implemented
!cat converted_python/HSVPanSharpening.py

#!/usr/bin/env python

# Import earthengine API
import ee
import ee.mapclient

# Initialise
ee.Initialize()

# HSV-based Pan-Sharpening.

# Grab a sample L8 image and pull out the RGB and pan bands.
image = ee.Image(ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')
  .filterDate('2017-01-01', '2017-12-31')
  .filterBounds(ee.Geometry.Point(-122.0808, 37.3947))
  .sort('CLOUD_COVER')
  .first());

rgb = image.select('B4', 'B3', 'B2');
pan = image.select('B8');

# Convert to HSV, swap in the pan band, and convert back to RGB.
huesat = rgb.rgbToHsv().select('hue', 'saturation');
upres = ee.Image.cat(huesat, pan).hsvToRgb();

# There are many fine places to look; here is one.  Comment
# this out if you want to twiddle knobs while panning around.
Map.setCenter(-122.0808, 37.3947, 14);

# Display before and after layers using the same vis parameters.
Map.addLayer(rgb, {max: 0.3}, 'Original');
Map.addLayer(upres, {max: 0.3}, 'Pansharpened');
